In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
from utils import calculate_returns, compute_atr, compute_macd
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

master_ticker_data = sp500.copy()
master_ticker_data.rename(columns={"Symbol":"ticker"}, inplace = True)
master_ticker_data['ticker'] = master_ticker_data['ticker'].str.replace('.', '-')

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-12-15'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-12-18 A        37.390388   40.700001   40.799999   40.110001   40.480000   
           AAPL     25.086512   28.162500   28.162500   27.665001   27.967501   
           ABBV     44.905315   67.919998   68.010002   67.199997   67.860001   
           ABT      37.827778   45.770000   45.790001   44.439999   44.709999   
           ACGL     19.606667   19.606667   19.633333   19.383333   19.563334   
...                       ...         ...         ...         ...         ...   
2024-12-13 XYL     120.599998  120.599998  121.639999  120.400002  120.930000   
           YUM     137.000000  137.000000  139.250000  136.449997  138.410004   
           ZBH     106.500290  106.739998  106.919998  105.339996  106.440002   
           ZBRA    403.589996  403.589996  408.890015  401.549988  407.299988   
           ZTS     178.179993  178.179993  181.850006  176.630005  178.979996   

Price                   volume  
date       ticker               
2014-12-18 A         3104100.0  
           AAPL    236024800.0  
           ABBV      7184700.0  
           ABT       6703700.0  
           ACGL      1897200.0  
...                        ...  
2024-12-13 XYL       1353600.0  
           YUM       1044300.0  
           ZBH       1195300.0  
           ZBRA       249200.0  
           ZTS       1650300.0  

[1230726 rows x 6 columns]

In [3]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

df['sharpe_ratio'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.sharpe_ratio(close=x))

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2014-12-18 A        37.390388   40.700001   40.799999   40.110001   40.480000   
           AAPL     25.086512   28.162500   28.162500   27.665001   27.967501   
           ABBV     44.905315   67.919998   68.010002   67.199997   67.860001   
           ABT      37.827778   45.770000   45.790001   44.439999   44.709999   
           ACGL     19.606667   19.606667   19.633333   19.383333   19.563334   
...                       ...         ...         ...         ...         ...   
2024-12-13 XYL     120.599998  120.599998  121.639999  120.400002  120.930000   
           YUM     137.000000  137.000000  139.250000  136.449997  138.410004   
           ZBH     106.500290  106.739998  106.919998  105.339996  106.440002   
           ZBRA    403.589996  403.589996  408.890015  401.549988  407.299988   
           ZTS     178.179993  178.179993  181.850006  176.630005  178.979996   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2014-12-18 A         3104100.0         -0.002290        NaN       NaN   
           AAPL    236024800.0         -0.004407        NaN       NaN   
           ABBV      7184700.0         -0.065783        NaN       NaN   
           ABT       6703700.0         -0.010345        NaN       NaN   
           ACGL      1897200.0          0.000080        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-13 XYL       1353600.0          0.000050  39.094811  4.795941   
           YUM       1044300.0          0.000166  51.749912  4.893467   
           ZBH       1195300.0          0.000111  45.108724  4.672756   
           ZBRA       249200.0          0.000132  56.606318  5.948302   
           ZTS       1650300.0          0.000416  48.957056  5.166558   

Price                bb_mid   bb_high  sharpe_ratio       atr      macd  \
date       ticker                                                         
2014-12-18 A            NaN       NaN      0.628146       NaN       NaN   
           AAPL         NaN       NaN      0.949969       NaN       NaN   
           ABBV         NaN       NaN      0.642419       NaN       NaN   
           ABT          NaN       NaN      0.584895       NaN       NaN   
           ACGL         NaN       NaN      0.721101       NaN       NaN   
...                     ...       ...           ...       ...       ...   
2024-12-13 XYL     4.838110  4.880278      0.613988  1.052145 -0.727624   
           YUM     4.926469  4.959472      0.608800  0.473350  0.476985   
           ZBH     4.706051  4.739346      0.148368 -0.521394 -0.139275   
           ZBRA    5.995411  6.042521      0.617572  0.112353  0.663372   
           ZTS     5.179572  5.192586      0.705045  0.686407 -0.416899   

Price              dollar_volume  
date       ticker                 
2014-12-18 A          116.063505  
           AAPL      5921.038886  
           ABBV       322.631220  
           ABT        253.586074  
           ACGL        37.197768  
...                          ...  
2024-12-13 XYL        163.244158  
           YUM        143.069100  
           ZBH        127.299797  
           ZBRA       100.574627  
           ZTS        294.050442  

[1230726 rows x 15 columns]

In [4]:
data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-01-27 A        35.686131   38.750000   39.250000   38.580002   38.700001   
           AAPL     24.304857   27.285000   28.120001   27.257500   28.105000   
           ABBV     42.031761   63.099998   63.500000   62.200001   62.330002   
           ABT      36.291519   43.680000   44.020000   43.430000   43.880001   
           ACGL     19.953333   19.953333   20.160000   19.683332   19.760000   
...                       ...         ...         ...         ...         ...   
2024-12-13 XYL     120.599998  120.599998  121.639999  120.400002  120.930000   
           YUM     137.000000  137.000000  139.250000  136.449997  138.410004   
           ZBH     106.500290  106.739998  106.919998  105.339996  106.440002   
           ZBRA    403.589996  403.589996  408.890015  401.549988  407.299988   
           ZTS     178.179993  178.179993  181.850006  176.630005  178.979996   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-01-27 A         1703500.0         -0.002391  42.551096  3.570881   
           AAPL    382274800.0         -0.007667  45.914721  3.195314   
           ABBV      6532300.0         -0.059758  41.454675  3.738411   
           ABT       4847400.0         -0.013836  39.157173  3.615258   
           ACGL      1379700.0          0.000250  58.501950  3.014891   
...                        ...               ...        ...       ...   
2024-12-13 XYL       1353600.0          0.000050  39.094811  4.795941   
           YUM       1044300.0          0.000166  51.749912  4.893467   
           ZBH       1195300.0          0.000111  45.108724  4.672756   
           ZBRA       249200.0          0.000132  56.606318  5.948302   
           ZTS       1650300.0          0.000416  48.957056  5.166558   

Price                bb_mid  ...  sharpe_ratio       atr      macd  \
date       ticker            ...                                     
2015-01-27 A       3.622549  ...      0.628146 -1.001110 -0.615512   
           AAPL    3.237558  ...      0.949969 -0.921370 -0.295066   
           ABBV    3.780835  ...      0.642419 -0.570094 -0.687059   
           ABT     3.638065  ...      0.584895 -1.015763 -0.567776   
           ACGL    3.030217  ...      0.721101 -1.057993 -0.175617   
...                     ...  ...           ...       ...       ...   
2024-12-13 XYL     4.838110  ...      0.613988  1.052145 -0.727624   
           YUM     4.926469  ...      0.608800  0.473350  0.476985   
           ZBH     4.706051  ...      0.148368 -0.521394 -0.139275   
           ZBRA    5.995411  ...      0.617572  0.112353  0.663372   
           ZTS     5.179572  ...      0.705045  0.686407 -0.416899   

Price              dollar_volume  return_1d  return_2d  return_3d  return_6d  \
date       ticker                                                              
2015-01-27 A           60.791323  -0.010217  -0.000773  -0.007624   0.002167   
           AAPL      9291.134446  -0.035013  -0.017141  -0.009763   0.004893   
           ABBV       274.564073   0.004297   0.005352   0.000635  -0.003754   
           ABT        175.919510  -0.010646  -0.002054  -0.006106  -0.003058   
           ACGL        27.529613   0.002176   0.003780   0.001060   0.001709   
...                          ...        ...        ...        ...        ...   
2024-12-13 XYL        163.244158  -0.003882  -0.011330  -0.021106  -0.007576   
           YUM        143.069100  -0.011829  -0.005573  -0.002951  -0.001368   
           ZBH        127.299797  -0.000936  -0.000795  -0.005131  -0.003729   
           ZBRA       100.574627  -0.009935  -0.000718   0.001499  -0.000573   
           ZTS        294.050442  -0.003690   0.002846   0.002765   0.003226   

Price              return_9d  return_12d  
d

#### Download Fama-French Factors and Calculate Rolling Factor Betas.

We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.

In [5]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1d']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1d
date       ticker                                                   
2015-03-31 A      -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.004075
           AAPL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015352
           ABBV   -0.0112  0.0307 -0.0038  0.0009 -0.0052   0.005151
           ABT    -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015930
           ACGL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.017230
...                   ...     ...     ...     ...     ...        ...
2024-10-31 XYL    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.057777
           YUM    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.016718
           ZBH    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.026318
           ZBRA   -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.005001
           ZTS    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.019352

[40613 rows x 6 columns]

In [6]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1d
date       ticker                                                   
2015-03-31 A      -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.004075
           AAPL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015352
           ABBV   -0.0112  0.0307 -0.0038  0.0009 -0.0052   0.005151
           ABT    -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.015930
           ACGL   -0.0112  0.0307 -0.0038  0.0009 -0.0052  -0.017230
...                   ...     ...     ...     ...     ...        ...
2024-10-31 XYL    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.057777
           YUM    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.016718
           ZBH    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.026318
           ZBRA   -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.005001
           ZTS    -0.0097 -0.0088  0.0089 -0.0138  0.0103  -0.019352

[40595 rows x 6 columns]

Calculate Rolling Factor Betas.

In [7]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1d'], 
                                     exog=sm.add_constant(x.drop('return_1d', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2015-03-31 A            NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACGL         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-10-31 XYL     0.145636 -0.002475  0.044490  0.244988 -0.410126
           YUM     0.086272  0.056704  0.021998  0.210426 -0.131479
           ZBH     0.169773 -0.132425  0.044182  0.011428 -0.142852
           ZBRA    0.076823  0.158274 -0.055864  0.140401 -0.089678
           ZTS     0.087858  0.047315 -0.143145  0.203833  0.043834

[40595 rows x 5 columns]

Join the rolling factors data to the main features dataframe

In [8]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1215911 entries, (Timestamp('2015-01-27 00:00:00'), 'A') to (Timestamp('2024-12-13 00:00:00'), 'ZTS')
Data columns (total 25 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   close             1215911 non-null  float64
 1   high              1215911 non-null  float64
 2   low               1215911 non-null  float64
 3   open              1215911 non-null  float64
 4   volume            1215911 non-null  float64
 5   garman_klass_vol  1215911 non-null  float64
 6   rsi               1215911 non-null  float64
 7   bb_low            1215911 non-null  float64
 8   bb_mid            1215911 non-null  float64
 9   bb_high           1215911 non-null  float64
 10  sharpe_ratio      1215911 non-null  float64
 11  atr               1215911 non-null  float64
 12  macd              1215911 non-null  float64
 13  dollar_volume     1215911 non-null  float64
 14  return_1d         1215911 

In [9]:
data.head()

close       high        low       open       volume  \
date       ticker                                                            
2015-01-27 A       38.750000  39.250000  38.580002  38.700001    1703500.0   
           AAPL    27.285000  28.120001  27.257500  28.105000  382274800.0   
           ABBV    63.099998  63.500000  62.200001  62.330002    6532300.0   
           ABT     43.680000  44.020000  43.430000  43.880001    4847400.0   
           ACGL    19.953333  20.160000  19.683332  19.760000    1379700.0   

                   garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2015-01-27 A              -0.002391  42.551096  3.570881  3.622549  3.674216   
           AAPL           -0.007667  45.914721  3.195314  3.237558  3.279801   
           ABBV           -0.059758  41.454675  3.738411  3.780835  3.823258   
           ABT            -0.013836  39.157173  3.615258  3.638065  3.660871   
           ACGL            0.000250  58.501950  3.014891  3.030217  3.045544   

                   ...  return_2d  return_3d  return_6d  return_9d  \
date       ticker  ...                                               
2015-01-27 A       ...  -0.000773  -0.007624   0.002167  -0.002268   
           AAPL    ...  -0.017141  -0.009763   0.004893  -0.001093   
           ABBV    ...   0.005352   0.000635  -0.003754  -0.000509   
           ABT     ...  -0.002054  -0.006106  -0.003058  -0.002338   
           ACGL    ...   0.003780   0.001060   0.001709   0.001308   

                   return_12d    Mkt-RF       SMB       HML       RMW  \
date       ticker                                                       
2015-01-27 A        -0.004470 -0.013231 -0.040100  0.049201  0.050533   
           AAPL     -0.002072  0.073535 -0.020395 -0.015089  0.155134   
           ABBV     -0.005141  0.072633  0.045399 -0.073540  0.175087   
           ABT      -0.003286 -0.030629  0.048402  0.014878  0.151122   
           ACGL      0.000321 -0.064490 -0.056543  0.171240 -0.082922   

                        CMA  
date       ticker            
2015-01-27 A      -0.007434  
           AAPL    0.208846  
           ABBV   -0.034091  
           ABT    -0.068622  
           ACGL   -0.165375  

[5 rows x 25 columns]

### Save price data

In [10]:
data.reset_index(inplace = True)

In [11]:
data.to_parquet("data/price_data.gzip", compression='gzip', index = False)
data.to_csv("data/price_data.csv", index = False)

: 

### Save data to sql db

In [12]:
import sqlite3
conn = sqlite3.connect('data/database.db')
c = conn.cursor()

In [13]:
data.to_sql('price_data', conn, if_exists='replace', index = False)
master_ticker_data.to_sql('master_ticker', conn, if_exists='replace')

502